## Credit Card Fraud Dataset

# define global variable

In [ ]:
random_state=1980
sampling_strategy=1.0

# Import dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Reshape, Dropout,BatchNormalization,Conv1DTranspose,Conv1D,Flatten
import tensorflow as tf
from tensorflow.keras.layers import LeakyReLU

In [ ]:
data = pd.read_csv("/content/creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
data.Class.value_counts()

,count
Class,
0,284315
1,492


In [ ]:
var_list=data.columns[:-1]

In [ ]:
print(var_list)

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
      dtype='object')


# data preparation

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(data,test_size=0.2,random_state=random_state)

In [ ]:
print(f"the shape of training dataset is {train.shape}")
print(f"the shape of testing dataset is {test.shape}")

the shape of training dataset is (227845, 31)
the shape of testing dataset is (56962, 31)


In [ ]:
X_train,y_train=train.iloc[:,:-1],train.iloc[:,-1]
X_test,y_test=test.iloc[:,:-1],test.iloc[:,-1]

In [ ]:
majority_class=train.Class.value_counts()[0]
minority_class=train.Class.value_counts()[1]
print(f"the amount of majority class:{majority_class}")
print(f"the amount of minority class:{minority_class}")

the amount of majority class:227457
the amount of minority class:388


# data normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
X_train_sc= scaler.fit_transform(X_train)
X_test_sc=scaler.transform(X_test)

In [ ]:
print(f"train dataset:\nmax:{X_train_sc.max()}\nmin:{X_train_sc.min()}\nmean:{X_train_sc.mean()}\nstd:{X_train_sc.std()}")
print(f"\ntest dataset:\nmax:{X_test_sc.max()}\nmin:{X_test_sc.min()}\nmean:{X_test_sc.mean()}\nstd:{X_test_sc.std()}")

train dataset:
max:1.0
min:0.0
mean:0.5172471291571057
std:0.21914915033922755

test dataset:
max:1.1632934326709123
min:-0.21670863808324825
mean:0.5172103984074053
std:0.21913448081838327


In [ ]:
tx_ori,vx_ori,ty_ori,vy_ori=train_test_split(X_train_sc,y_train,test_size=0.2,random_state=random_state)
print(f"the shape of tx_vgan is {tx_ori.shape}")
print(f"the shape of ty_vgan is {vx_ori.shape}")
print(f"the shape of vx_vgan is {ty_ori.shape}")
print(f"the shape of vy_vgan is {vy_ori.shape}")

the shape of tx_vgan is (182276, 30)
the shape of ty_vgan is (45569, 30)
the shape of vx_vgan is (182276,)
the shape of vy_vgan is (45569,)


# GANs

In [ ]:
original_data = pd.concat([pd.DataFrame(X_train_sc, columns=X_train.columns, index=X_train.index),
                           pd.DataFrame(y_train, columns=["Class"], index=y_train.index)], axis=1)

In [ ]:
vanilla_generator=tf.keras.models.load_model('vanilla_generator_'+str(random_state)+'.h5')

In [ ]:
# generate a batch of noisy input
def sythetic_data(generator,num_dt,random_normal_dimensinos):
  noise=tf.random.normal(shape=(num_dt,random_normal_dimensinos))
  sythetic_data=generator(noise)
  return sythetic_data

In [ ]:
generator_num=int(sampling_strategy*majority_class)-minority_class
random_normal_dimensinos=100

vanilla_gan_data=sythetic_data(vanilla_generator,generator_num,random_normal_dimensinos)
vgan_dt=pd.DataFrame(vanilla_gan_data.numpy(),columns=var_list)
vgan_dt["Class"]=np.ones(generator_num)

In [ ]:
vgan_train=pd.concat([original_data,vgan_dt],axis=0)
vgan_train.to_csv("vgan_sampling_"+str(sampling_strategy)+".csv")

In [ ]:
X_vgan,y_vgan=np.array(vgan_train.iloc[:,:-1]),np.array(vgan_train.iloc[:,-1])

In [ ]:
print(f"the shape of X_vgan is {X_vgan.shape}")
print(f"the shape of y_vgan is {y_vgan.shape}")
print(f"the number of minority class is {np.sum(y_vgan==1)}")
print(f"the number of majority class is {np.sum(y_vgan==0)}")

the shape of X_vgan is (454914, 30)
the shape of y_vgan is (454914,)
the number of minority class is 227457
the number of majority class is 227457


In [ ]:
tx_vgan,vx_vgan,ty_vgan,vy_vgan=train_test_split(X_vgan,y_vgan,test_size=0.1,random_state=random_state)
print(f"the shape of tx_vgan is {tx_vgan.shape}")
print(f"the shape of ty_vgan is {ty_vgan.shape}")
print(f"the shape of vx_vgan is {vx_vgan.shape}")
print(f"the shape of vy_vgan is {vy_vgan.shape}")

the shape of tx_vgan is (409422, 30)
the shape of ty_vgan is (409422,)
the shape of vx_vgan is (45492, 30)
the shape of vy_vgan is (45492,)


# SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy=sampling_strategy,random_state=random_state)
X_smote,y_smote= smote.fit_resample(X_train_sc,y_train)
X_smote,y_smote=np.array(X_smote),np.array(y_smote)

In [ ]:
print(f"the shape of X_smote is {X_smote.shape}")
print(f"the shape of y_smote is {y_smote.shape}")
print(f"the number of minority class is {np.sum(y_smote==1)}")
print(f"the number of majority class is {np.sum(y_smote==0)}")

the shape of X_smote is (454914, 30)
the shape of y_smote is (454914,)
the number of minority class is 227457
the number of majority class is 227457


In [ ]:
tx_sm,vx_sm,ty_sm,vy_sm=train_test_split(X_smote,y_smote,test_size=0.1,random_state=random_state)
print(f"the shape of tx_sm is {tx_sm.shape}")
print(f"the shape of ty_sm is {ty_sm.shape}")
print(f"the shape of vx_sm is {vx_sm.shape}")
print(f"the shape of vy_sm is {vy_sm.shape}")


the shape of tx_sm is (409422, 30)
the shape of ty_sm is (409422,)
the shape of vx_sm is (45492, 30)
the shape of vy_sm is (45492,)


# ADASYN

In [ ]:
from imblearn.over_sampling import ADASYN
adasyn = ADASYN(sampling_strategy=sampling_strategy,random_state=random_state)
X_adasyn,y_adasyn= adasyn.fit_resample(X_train_sc,y_train)
X_adasyn,y_adasyn=np.array(X_adasyn),np.array(y_adasyn)

In [ ]:
print(f"the shape of X_adasyn is {X_adasyn.shape}")
print(f"the shape of y_adasyn is {y_adasyn.shape}")
print(f"the number of minority class is {np.sum(y_adasyn==1)}")
print(f"the number of majority class is {np.sum(y_adasyn==0)}")

the shape of X_adasyn is (454972, 30)
the shape of y_adasyn is (454972,)
the number of minority class is 227515
the number of majority class is 227457


In [ ]:
tx_ad,vx_ad,ty_ad,vy_ad=train_test_split(X_adasyn,y_adasyn,test_size=0.1,random_state=random_state)
print(f"the shape of tx_sm is {tx_ad.shape}")
print(f"the shape of ty_sm is {ty_ad.shape}")
print(f"the shape of vx_sm is {vx_ad.shape}")
print(f"the shape of vy_sm is {vy_ad.shape}")

the shape of tx_sm is (409474, 30)
the shape of ty_sm is (409474,)
the shape of vx_sm is (45498, 30)
the shape of vy_sm is (45498,)


# Build a model

## define functions

In [ ]:
# define a function to evaluate models by accuracy, precision, recall and f1
from sklearn.metrics import recall_score,accuracy_score,precision_score,f1_score
def display_metrics(Y,Y_predictions):

    precision = precision_score(Y,Y_predictions)
    recall= recall_score(Y,Y_predictions)
    accuracy = accuracy_score(Y,Y_predictions)
    f1 = f1_score(Y,Y_predictions)
    print(f"Accuracy：{accuracy:.5f}")
    print(f"Precision：{precision:.5f}")
    print(f"Recall：{recall:.5f}")
    print(f"F1：{f1:.5f}")
    print("\n")

    results={}
    results["accuracy"]=accuracy
    results["precision"]=precision
    results["recall"]=recall
    results["f1"]=f1

    return results

In [ ]:
#define a function to finetune models using GridSearchCV
from sklearn.model_selection import GridSearchCV

def Finetune_CV(clf,param_grid,metric,cv_num,X_train,y_train):
    grid_search = GridSearchCV(clf, param_grid, cv=cv_num,
                           scoring=metric, return_train_score=True)
    grid_search.fit(X_train,y_train)
    print(f"{metric}_best_params:{grid_search.best_params_}")
    cvres = grid_search.cv_results_
    print(f"mean_{metric} and params:")
    for score, params in zip(cvres["mean_test_score"], cvres["params"]):
        print(f"{score:.4f},{params}")
    print("\n")

In [ ]:
#define a function to display_confusion_matrix
from sklearn.metrics import confusion_matrix

def display_confusion_matrix(y_test,y_prediction):
    cm = confusion_matrix(y_test,y_prediction)
    cm_df = pd.DataFrame(cm, index=['ActNeg', 'ActPos'], columns=['PredNeg', 'PredPos'])
    # draw heatmap
    plt.figure(figsize=(3, 2))
    sns.set(font_scale=1)
    sns.heatmap(cm_df, annot=True, cmap='Blues', fmt='g', annot_kws={"size": 12})
    plt.title('Confusion Matrix')
    plt.xlabel('PredLabel')
    plt.ylabel('TrueLabel')
    plt.show()

In [ ]:
# define a function to build initial models together

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score,classification_report,confusion_matrix

def add_prefix_to_keys(original_dict, prefix):
    updated_dict = {}
    for key, value in original_dict.items():
        new_key = prefix + key  # Add the prefix to the original key
        updated_dict[new_key] = value  # Update the dictionary with the new key-value pair
    return updated_dict

def initial_model(model,X_train,y_train,X_test,y_test,datasettyp=['train','val']):

    m=model.fit(X_train,y_train)
    pred_train=m.predict(X_train)
    pred_test=m.predict(X_test)

    print(datasettyp[0]+"_metrics")
    train_results=display_metrics(y_train,pred_train)


    print(datasettyp[1]+"_metrics")
    test_results=display_metrics(y_test,pred_test)

    #print(confusion_matrix(y_test,pred_test))

    results=dict(add_prefix_to_keys(train_results, datasettyp[0]+"_"),**add_prefix_to_keys(test_results, datasettyp[1]+"_"))

    return m,results

def eval_model(model,X_test,y_test):
    pred_test=model.predict(X_test)
    print("test_metrics")
    test_results=display_metrics(y_test,pred_test)
    results=add_prefix_to_keys(test_results, "test_")
    print(confusion_matrix(y_test,pred_test))
    print("\n")
    return results


In [ ]:
def write_to_csv(model_name, metrics_dict,classifier):
# Write metrics to CSV file
  model_names = model_name
  model_metrics_lists = metrics_dict

# Prepare data for DataFrame in the right format
  data = []
  for model_name, metrics in zip(model_names, model_metrics_lists):
      for metric, value in metrics.items():
          data.append({'Model': model_name, 'Metric': metric, 'Value': value})

# Create Pandas DataFrame
  metrics_df = pd.DataFrame(data)
  metrics_df.set_index(['Model', 'Metric'], inplace=True)

# Write DataFrame to CSV with multiple sheets
  writer = pd.ExcelWriter(classifier_name+'_metrics.xlsx')
  for model_name, model_metrics_df in metrics_df.groupby('Model'):
      model_metrics_df.to_excel(writer, sheet_name=model_name)
  writer.close() # Use .close() to save and close the Excel file

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(random_state=random_state)

print("the experiment results of original dataset:\n")
m1,m1_val=initial_model(model,tx_ori,ty_ori,vx_ori,vy_ori)
m1_test=eval_model(m1,X_test_sc,y_test)

print("the experiment results of vgan:\n")
m3,m3_val=initial_model(model,tx_vgan,ty_vgan,vx_vgan,vy_vgan)
m3_test=eval_model(m3,X_test_sc,y_test)

print("the experiment results of smote:\n")
m4,m4_val=initial_model(model,tx_sm,ty_sm,vx_sm,vy_sm)
m4_test=eval_model(m4,X_test_sc,y_test)

print("the experiment results of adasyn:\n")
m5,m5_val=initial_model(model,tx_ad,ty_ad,vx_ad,vy_ad)
m5_test=eval_model(m5,X_test_sc,y_test)

m1_results=dict(m1_val,**m1_test)
m3_results=dict(m3_val,**m3_test)
m4_results=dict(m4_val,**m4_test)
m5_results=dict(m5_val,**m5_test)

model_metrics_lists = [m1_results, m3_results, m4_results, m5_results]
model_names = ['original', 'vgan', 'smote', 'adasyn']

classifier_name="LogisticRegression"
write_to_csv(model_names, model_metrics_lists,classifier_name)

the experiment results of original dataset:

train_metrics
Accuracy：0.99896
Precision：0.84444
Recall：0.48562
F1：0.61663


val_metrics
Accuracy：0.99921
Precision：0.89796
Recall：0.58667
F1：0.70968


test_metrics
Accuracy：0.99914
Precision：0.88732
Recall：0.60577
F1：0.72000


[[56850     8]
 [   41    63]]


the experiment results of vgan:

train_metrics
Accuracy：0.99954
Precision：0.99985
Recall：0.99923
F1：0.99954


val_metrics
Accuracy：0.99963
Precision：0.99982
Recall：0.99943
F1：0.99963


test_metrics
Accuracy：0.99926
Precision：0.89744
Recall：0.67308
F1：0.76923


[[56850     8]
 [   34    70]]


the experiment results of smote:

train_metrics
Accuracy：0.94711
Precision：0.97599
Recall：0.91675
F1：0.94544


val_metrics
Accuracy：0.94687
Precision：0.97525
Recall：0.91712
F1：0.94529


test_metrics
Accuracy：0.97646
Precision：0.06657
Recall：0.91346
F1：0.12410


[[55526  1332]
 [    9    95]]


the experiment results of adasyn:

train_metrics
Accuracy：0.89195
Precision：0.91855
Recall：0.86035
F1：0.8

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(random_state=random_state,criterion='entropy',max_depth=4,min_samples_leaf=10,n_estimators=100)

print("the experiment results of original dataset:\n")
m1,m1_val=initial_model(model,tx_ori,ty_ori,vx_ori,vy_ori)
m1_test=eval_model(m1,X_test_sc,y_test)

print("the experiment results of vgan:\n")
m3,m3_val=initial_model(model,tx_vgan,ty_vgan,vx_vgan,vy_vgan)
m3_test=eval_model(m3,X_test_sc,y_test)

print("the experiment results of smote:\n")
m4,m4_val=initial_model(model,tx_sm,ty_sm,vx_sm,vy_sm)
m4_test=eval_model(m4,X_test_sc,y_test)

print("the experiment results of adasyn:\n")
m5,m5_val=initial_model(model,tx_ad,ty_ad,vx_ad,vy_ad)
m5_test=eval_model(m5,X_test_sc,y_test)

m1_results=dict(m1_val,**m1_test)
m3_results=dict(m3_val,**m3_test)
m4_results=dict(m4_val,**m4_test)
m5_results=dict(m5_val,**m5_test)

model_metrics_lists = [m1_results, m3_results, m4_results, m5_results]
model_names = ['original', 'vgan', 'smote', 'adasyn']

classifier_name="RandomForestClassifier"
write_to_csv(model_names, model_metrics_lists,classifier_name)

the experiment results of original dataset:

train_metrics
Accuracy：0.99945
Precision：0.91732
Recall：0.74441
F1：0.82187


val_metrics
Accuracy：0.99954
Precision：0.93548
Recall：0.77333
F1：0.84672


test_metrics
Accuracy：0.99942
Precision：0.89888
Recall：0.76923
F1：0.82902


[[56849     9]
 [   24    80]]


the experiment results of vgan:

train_metrics
Accuracy：0.99957
Precision：0.99984
Recall：0.99931
F1：0.99957


val_metrics
Accuracy：0.99958
Precision：0.99974
Recall：0.99943
F1：0.99958


test_metrics
Accuracy：0.99916
Precision：0.86842
Recall：0.63462
F1：0.73333


[[56848    10]
 [   38    66]]


the experiment results of smote:

train_metrics
Accuracy：0.93333
Precision：0.99619
Recall：0.86997
F1：0.92881


val_metrics
Accuracy：0.93337
Precision：0.99673
Recall：0.86974
F1：0.92891


test_metrics
Accuracy：0.99626
Precision：0.31399
Recall：0.88462
F1：0.46348


[[56657   201]
 [   12    92]]


the experiment results of adasyn:

train_metrics
Accuracy：0.88757
Precision：0.95204
Recall：0.81644
F1：0.8

## xgboost

In [ ]:
from xgboost import XGBClassifier

model=XGBClassifier(random_state=random_state,n_estimators=100,learning_rate=0.1)

print("the experiment results of original dataset:\n")
m1,m1_val=initial_model(model,tx_ori,ty_ori,vx_ori,vy_ori)
m1_test=eval_model(m1,X_test_sc,y_test)

print("the experiment results of vgan:\n")
m3,m3_val=initial_model(model,tx_vgan,ty_vgan,vx_vgan,vy_vgan)
m3_test=eval_model(m3,X_test_sc,y_test)

print("the experiment results of smote:\n")
m4,m4_val=initial_model(model,tx_sm,ty_sm,vx_sm,vy_sm)
m4_test=eval_model(m4,X_test_sc,y_test)

print("the experiment results of adasyn:\n")
m5,m5_val=initial_model(model,tx_ad,ty_ad,vx_ad,vy_ad)
m5_test=eval_model(m5,X_test_sc,y_test)

m1_results=dict(m1_val,**m1_test)
m3_results=dict(m3_val,**m3_test)
m4_results=dict(m4_val,**m4_test)
m5_results=dict(m5_val,**m5_test)

model_metrics_lists = [m1_results, m3_results, m4_results, m5_results]
model_names = ['original', 'vgan', 'smote', 'adasyn']

classifier_name="XGBClassifier"
write_to_csv(model_names, model_metrics_lists,classifier_name)

the experiment results of original dataset:

train_metrics
Accuracy：0.99990
Precision：1.00000
Recall：0.94249
F1：0.97039


val_metrics
Accuracy：0.99967
Precision：0.96875
Recall：0.82667
F1：0.89209


test_metrics
Accuracy：0.99961
Precision：0.94565
Recall：0.83654
F1：0.88776


[[56853     5]
 [   17    87]]


the experiment results of vgan:

train_metrics
Accuracy：0.99990
Precision：1.00000
Recall：0.99981
F1：0.99990


val_metrics
Accuracy：0.99976
Precision：0.99978
Recall：0.99974
F1：0.99976


test_metrics
Accuracy：0.99960
Precision：0.94505
Recall：0.82692
F1：0.88205


[[56853     5]
 [   18    86]]


the experiment results of smote:

train_metrics
Accuracy：0.99874
Precision：0.99777
Recall：0.99971
F1：0.99874


val_metrics
Accuracy：0.99864
Precision：0.99763
Recall：0.99965
F1：0.99864


test_metrics
Accuracy：0.99680
Precision：0.35115
Recall：0.88462
F1：0.50273


[[56688   170]
 [   12    92]]


the experiment results of adasyn:

train_metrics
Accuracy：0.99788
Precision：0.99582
Recall：0.99996
F1：0.9